In [1]:
import time
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine # database connection
import datetime as dt
from IPython.display import display

import plotly.plotly as py # interactive graphing
from plotly.graph_objs import Bar, Scatter, Marker, Layout 

from calc import calc_x_cy,calc_y_cy

In [2]:
colnames1={
'tripduration':'duration',
'starttime':'starttime',
'stoptime':'stoptime',
'start station id':'stn1_id',
'start station name':'stn1_name',
'start station latitude':'lat1',
'start station longitude':'long1',
'end station id':'stn2_id',
'end station name':'stn2_name',
'end station latitude':'lat2',
'end station longitude':'long2',
'bikeid':'bikeid',
'usertype':'usertype',
'birth year':'birth_yr',
'gender':'gender',

}

colnames2={
'Trip Duration':'duration',
'Start Time':'starttime',
'Stop Time':'stoptime',
'Start Station ID':'stn1_id',
'Start Station Name':'stn1_name',
'Start Station Latitude':'lat1',
'Start Station Longitude':'long1',
'End Station ID':'stn2_id',
'End Station Name':'stn2_name',
'End Station Latitude':'lat2',
'End Station Longitude':'long2',
'Bike ID':'bikeid',
'User Type':'usertype',
'Birth Year':'birth_yr',
'Gender':'gender'
}

lat_min,lat_max=40.645,40.965 #0.16
long_min,long_max=-74.020,-73.830 #0.095

lat_sep=20
long_sep=20
lat_div=(lat_max-lat_min)/lat_sep
long_div=(long_max-long_min)/long_sep

## 1. Take in trip count raw file for each month and aggregate by grid/month

In [3]:
disk_engine = create_engine('sqlite:///trip.db')

In [ ]:
files=[f for f in os.listdir('trip/') if 'data' in f and '.csv' in f]
print (files,'\n','\n')

very_start=time.time()

for eachFile in files:
    year=int(eachFile[0:4])
    if eachFile[4]=='-':
        month=int(eachFile[5:7])
    else:
        month=int(eachFile[4:6])
    
    if year>2014:
        continue
    if year==2014 and month >=8:
        continue
    print(eachFile)
    
    df=pd.read_csv('trip/'+eachFile,sep=',',low_memory=True,skipinitialspace=True)#,nrows=10000)
    if df.columns[0]=="tripduration":
            colnames=colnames1
    else:
        colnames=colnames2
    df.rename(columns=colnames,inplace=True)

    df=df[df.lat1.between(35,45)];df=df[df.long1.between(-80,-60)]
    df=df[df.lat2.between(35,45)];df=df[df.long2.between(-80,-60)]

            #     columns=['duration','starttime','stn1_id','stn1_name','lat1','long1'
            #             ,'birth_yr','gender']
            #     for c in df.columns:
            #         if c not in columns:
            #             df = df.drop(c, axis=1)
    
    #------- Add in time paramters -------
    df.starttime=pd.to_datetime(df.starttime)
    df['yr']=df.starttime.dt.year
    df['month']=df.starttime.dt.month
    df['week']=df.starttime.dt.week
    df['hr']=df.starttime.dt.hour
    df['minute']=df.starttime.dt.minute
    df['weekday']=df.starttime.dt.weekday
    df['weekend']=0
    df.loc[(df.weekday==5)|(df.weekday==6),'weekend']=1
    df['time_prd']=df.hr
    df['count']=1
    df['grid_x']=calc_x_cy(df['long1'].values)
    df['grid_y']=calc_y_cy(df['lat1'].values)
    
                        #     # Modify some week number
                        #     df_temp=df[(df.month==12) & (df.week==1)]
                        #     df_temp['yr']=df_temp['yr']+1
                        #     df[(df.month==12) & (df.week==1)]=df_temp
                        #     df_temp=df[(df.month==1) & (df.week>=50)]
                        #     df_temp['yr']=df_temp['yr']-1
                        #     df[(df.month==1) & (df.week>=50)]=df_temp
    
    #------- Remove some strange characters in stn_name and birth_yr ------
    names=df.stn1_name.values
    for i,name in enumerate(names):
        names[i]=name.replace('\xa0',' ')
    df.stn1_name=names
    
    #------- Calculate age ------
    df=df[df['birth_yr'].notnull()]
    try:
        df=df[(df['birth_yr']>='0')&(df['birth_yr']<='2')]
        df['age']=df.yr-df.birth_yr.astype(int)
    except:
        df=df[df.birth_yr>1800]
        df['age']=df.yr-df.birth_yr
        
    

    #------- Trip count by time period --------
    df1=pd.pivot_table(df,index=['yr','month','grid_x','grid_y'],
                   values=['count'],columns='hr',fill_value=0,aggfunc=np.sum)
    df1.columns=df1.columns.droplevel()
    df1.reset_index(level=[0,1,2,3],inplace=True)
    df2=df.groupby(['grid_x','grid_y','yr','month'],as_index=False).agg({'count':'count','age':'mean','duration':'mean'})
    df3=pd.merge(df1,df2,on=['grid_x','grid_y','yr','month'],how='outer')
    df3.to_csv('trip_grid/'+eachFile,index=False)
    
    end=time.time()
    print(end-very_start)


## 2. Then summarize all the months into a single csv file 

In [4]:
files=[f for f in os.listdir('trip_grid/') if '.csv' in f]
dfs=[]
for eachFile in files:
#     print (eachFile)
    df=pd.read_csv('trip_grid/'+eachFile)
    dfs.append(df)
df=pd.concat(dfs)
df.to_csv('sum/trip_sum.csv',index=False)


In [5]:
df_x=df.groupby(['yr','month'],as_index=False).agg('sum')
df_x.to_csv('sum/trip_by_time.csv')
# print (df_x[0:5])

### Get the list of all CitiBike grids

In [ ]:
df1=df.groupby(['grid_x','grid_y']).agg('sum')
# df.columns=df.columns.droplevel()
df1.reset_index(level=[0,1],inplace=True)
df1=df1[['grid_x','grid_y','count']]
df1['citigrid']=1
df1.to_csv('sum/citibike_grid.csv',index=False) ## List of citibike grids
# print (df1[0:10])
# print (len(df1))


In [ ]:
# df=pd.read_csv('sum/trip_sum.csv')
# df1=df.groupby(['grid_x','grid_y']).agg('sum')
# # print (df[0:10])
# print (df1.columns)
# # df.columns=df.columns.droplevel()
# df1.reset_index(level=[0,1],inplace=True)
# df1=df1[['grid_x','grid_y']]
# df1['CitiBike']=1
# print (df1[0:10])
# print (len(df1))
# df1.to_csv('sum/citibike_grid.csv')

# df3=pd.merge(df,df1,on=['grid_x','grid_y'],how='outer')
# df3.to_csv('sum/trip_sum_new.csv',index=False)

## 4. Get the station list for each grid

In [ ]:
import time
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine # database connection
import datetime as dt
from IPython.display import display

import plotly.plotly as py # interactive graphing
from plotly.graph_objs import Bar, Scatter, Marker, Layout 

from calc import calc_x_cy,calc_y_cy

In [ ]:
colnames1={
'tripduration':'duration',
'starttime':'starttime',
'stoptime':'stoptime',
'start station id':'stn1_id',
'start station name':'stn1_name',
'start station latitude':'lat1',
'start station longitude':'long1',
'end station id':'stn2_id',
'end station name':'stn2_name',
'end station latitude':'lat2',
'end station longitude':'long2',
'bikeid':'bikeid',
'usertype':'usertype',
'birth year':'birth_yr',
'gender':'gender',

}

colnames2={
'Trip Duration':'duration',
'Start Time':'starttime',
'Stop Time':'stoptime',
'Start Station ID':'stn1_id',
'Start Station Name':'stn1_name',
'Start Station Latitude':'lat1',
'Start Station Longitude':'long1',
'End Station ID':'stn2_id',
'End Station Name':'stn2_name',
'End Station Latitude':'lat2',
'End Station Longitude':'long2',
'Bike ID':'bikeid',
'User Type':'usertype',
'Birth Year':'birth_yr',
'Gender':'gender'
}

lat_min,lat_max=40.645,40.965 #0.16
long_min,long_max=-74.020,-73.830 #0.095

lat_sep=20
long_sep=20
lat_div=(lat_max-lat_min)/lat_sep
long_div=(long_max-long_min)/long_sep

In [ ]:
files=[f for f in os.listdir('trip/') if 'data' in f and '.csv' in f]
print (files,'\n','\n')

for eachFile in files:
    if not '2013' in eachFile or not '7' in eachFile:
        continue
    year=int(eachFile[0:4])
    if eachFile[4]=='-':
        month=int(eachFile[5:7])
    else:
        month=int(eachFile[4:6])
    
    if year>2014:
        continue
    if year==2014 and month >=8:
        continue
    print(eachFile)
    
    df=pd.read_csv('trip/'+eachFile,sep=',',low_memory=True,skipinitialspace=True)#,nrows=10000)
    if df.columns[0]=="tripduration":
            colnames=colnames1
    else:
        colnames=colnames2
    df.rename(columns=colnames,inplace=True)

    df=df[df.lat1.between(35,45)];df=df[df.long1.between(-80,-60)]
    df=df[df.lat2.between(35,45)];df=df[df.long2.between(-80,-60)]

            #     columns=['duration','starttime','stn1_id','stn1_name','lat1','long1'
            #             ,'birth_yr','gender']
            #     for c in df.columns:
            #         if c not in columns:
            #             df = df.drop(c, axis=1)
    
    #------- Add in time paramters -------
    df.starttime=pd.to_datetime(df.starttime)
    df['yr']=df.starttime.dt.year
    df['month']=df.starttime.dt.month
    df['week']=df.starttime.dt.week
    df['hr']=df.starttime.dt.hour
    df['minute']=df.starttime.dt.minute
    df['weekday']=df.starttime.dt.weekday
    df['weekend']=0
    df.loc[(df.weekday==5)|(df.weekday==6),'weekend']=1
    df['time_prd']=df.hr
    df['count']=1
    df['grid_x']=calc_x_cy(df['long1'].values)
    df['grid_y']=calc_y_cy(df['lat1'].values)
    
                        #     # Modify some week number
                        #     df_temp=df[(df.month==12) & (df.week==1)]
                        #     df_temp['yr']=df_temp['yr']+1
                        #     df[(df.month==12) & (df.week==1)]=df_temp
                        #     df_temp=df[(df.month==1) & (df.week>=50)]
                        #     df_temp['yr']=df_temp['yr']-1
                        #     df[(df.month==1) & (df.week>=50)]=df_temp
    
    #------- Remove some strange characters in stn_name and birth_yr ------
    names=df.stn1_name.values
    for i,name in enumerate(names):
        names[i]=name.replace('\xa0',' ')
    df.stn1_name=names
    
    #------- Calculate age ------
    df=df[df['birth_yr'].notnull()]
    try:
        df=df[(df['birth_yr']>='0')&(df['birth_yr']<='2')]
        df['age']=df.yr-df.birth_yr.astype(int)
    except:
        df=df[df.birth_yr>1800]
        df['age']=df.yr-df.birth_yr

    #------- Trip count by time period --------
#     df1=pd.pivot_table(df,index=['yr','month','grid_x','grid_y'],
#                    values=['count'],columns='hr',fill_value=0,aggfunc=np.sum)
#     df1.columns=df1.columns.droplevel()
#     df1.reset_index(level=[0,1,2,3],inplace=True)
#     df2=df.groupby(['grid_x','grid_y','yr','month'],as_index=False).agg({'count':'count','age':'mean','duration':'mean'})
#     df3=pd.merge(df1,df2,on=['grid_x','grid_y','yr','month'],how='outer')
#     df3.to_csv('trip_grid/'+eachFile,index=False)
    df1=df.groupby(['stn1_name','stn1_id','lat1','long1','grid_x','grid_y'],as_index=False).agg({'count':'sum'})
    
    end=time.time()
    print(end-very_start)
    
cluster=pd.read_csv('sum/cluster.csv')
df1=pd.merge(df1,cluster,on=['grid_x','grid_y'],how='outer')
df1.to_csv('sum/grid_stns.csv',index=False)